<a href="https://colab.research.google.com/github/fecsaba/automatic_photo/blob/main/auto_get_meta.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drive**

In [1]:
import os

# Csatoljuk a Google Drive-ot
from google.colab import drive
drive.mount('/content/drive')
!pip install pypng
!pip install Pillow

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.0 MB/s eta 0:00:00


# ***Action***

```
'/content/drive/MyDrive/Images'
```



## Meta kiszedése

In [ ]:
from PIL import PngImagePlugin, Image
import os
import json

class PngValidator:
    """PNG fájlok érvényességének ellenőrzésére szolgáló osztály.

    Ez az osztály felelős a PNG fájlok formátumának validálásáért. Ellenőrzi, hogy egy fájl
    valóban érvényes PNG formátumú-e, függetlenül a fájl kiterjesztésétől.

    Methods:
        is_valid_png(file_path): Ellenőrzi egy fájl PNG formátumát.
    """

    @staticmethod
    def is_valid_png(file_path):
        """Ellenőrzi, hogy a fájl valóban PNG formátumú-e.

        A metódus megnyitja a fájlt és ellenőrzi annak belső formátumát,
        nem csak a kiterjesztést vizsgálja.

        Args:
            file_path (str): A vizsgálandó fájl teljes elérési útja.

        Returns:
            bool: True ha a fájl valódi PNG formátumú, False egyébként.

        Example:
            >>> validator = PngValidator()
            >>> is_valid = validator.is_valid_png("kep.png")
            >>> print(is_valid)
            True
        """
        try:
            with Image.open(file_path) as img:
                return isinstance(img, PngImagePlugin.PngImageFile) and img.format == 'PNG'
        except Exception:
            return False

class PngFileHandler:
    """PNG fájlok és mappák kezelésére szolgáló osztály.

    Ez az osztály felelős a fájlrendszer műveletek végrehajtásáért,
    beleértve a mappák beolvasását és a PNG fájlok azonosítását.

    Attributes:
        directory_path (str): A feldolgozandó mappa elérési útja.

    Methods:
        set_directory_path(directory_path): Beállítja a feldolgozandó mappa útvonalát.
        get_png_files(): Összegyűjti az érvényes PNG fájlokat a megadott mappából.
    """

    def __init__(self, directory_path=None):
        """Inicializálja a PngFileHandler osztályt.

        Args:
            directory_path (str, optional): A feldolgozandó mappa elérési útja.
                                          Alapértelmezett érték: None.
        """
        self.directory_path = directory_path

    def set_directory_path(self, directory_path):
        """Beállítja a feldolgozandó mappa útvonalát.

        Args:
            directory_path (str): A feldolgozandó mappa elérési útja.
        """
        self.directory_path = directory_path

    def get_png_files(self):
        """Összegyűjti az érvényes PNG fájlokat a megadott mappából.

        A metódus ellenőrzi a megadott mappában található összes fájlt,
        és csak azokat a fájlokat adja vissza, amelyek valóban PNG formátumúak.

        Returns:
            list: Érvényes PNG fájlok teljes elérési útjainak listája.

        Note:
            - Ha nincs megadva mappa útvonal, üres listával tér vissza
            - Ha a mappa nem létezik, üres listával tér vissza
            - Figyelmeztetést ad ki, ha talál olyan fájlt, ami .png kiterjesztésű,
              de nem valódi PNG formátumú
        """
        if not self.directory_path:
            print("Nincs megadva mappa útvonal!")
            return []

        if not os.path.exists(self.directory_path):
            print(f"A megadott mappa nem létezik: {self.directory_path}")
            return []

        png_files = []
        for filename in os.listdir(self.directory_path):
            file_path = os.path.join(self.directory_path, filename)
            if PngValidator.is_valid_png(file_path):
                png_files.append(file_path)
            elif filename.lower().endswith('.png'):
                print(f"\nFigyelmeztetés: A fájl '.png' kiterjesztésű, de nem valódi PNG: {filename}")

        return png_files

class MetadataReader:
    """PNG fájlok metaadatainak olvasására szolgáló osztály.

    Ez az osztály felelős a PNG fájlok metaadatainak kinyeréséért és
    megjelenítéséért. A metaadatok között szerepelhetnek például a készítő,
    a létrehozás dátuma, és egyéb egyedi információk.

    Methods:
        read_metadata(file_path): Beolvassa és megjeleníti egy PNG fájl metaadatait.
    """

    @staticmethod
    def read_metadata(file_path):
        """Beolvassa egy PNG fájl metaadatait.

        Args:
            file_path (str): A PNG fájl teljes elérési útja.

        Returns:
            dict: A metaadatok szótár formátumban, ahol a kulcsok a metaadat mezők nevei,
                 az értékek pedig a hozzájuk tartozó értékek.
                 Hiba esetén None-nal tér vissza.
                 Ha nincsenek metaadatok, üres szótárral tér vissza.

        Note:
            A metódus a következő eseteket kezeli:
            - Sikeres beolvasás esetén kiírja a metaadatokat
            - Ha nincsenek metaadatok, ezt jelzi
            - Ha a fájl nem található, hibaüzenetet ad
            - Egyéb hibák esetén hibaüzenetet ad
        """
        try:
            with Image.open(file_path) as img:
                metadata = img.text
                if metadata:
                    print(f"\nMetaadatok a következő fájlhoz: {os.path.basename(file_path)}")
                    print("-" * 50)
                    for k, v in metadata.items():
                        print(f"**{k}**:\n{v}\n")
                    return metadata
                else:
                    print(f"\nNem találtunk metaadatokat a következő fájlban: {os.path.basename(file_path)}")
                    return {}

        except FileNotFoundError:
            print(f"A fájl nem található: {file_path}")
            return None
        except Exception as e:
            print(f"Hiba történt a következő fájlnál {os.path.basename(file_path)}: {str(e)}")
            return None

class PngMetadataProcessor:
    """PNG fájlok metaadatainak feldolgozására szolgáló fő osztály.

    Ez az osztály koordinálja a teljes feldolgozási folyamatot, összekapcsolva
    a fájlkezelést és a metaadat-olvasást. Egy megadott mappában található összes
    PNG fájlt feldolgozza és összegyűjti azok metaadatait.

    Attributes:
        file_handler (PngFileHandler): A fájlkezelésért felelős objektum.

    Methods:
        process_directory(): Feldolgozza a megadott mappában található összes PNG fájlt.
        save_results_to_json(results): Elmenti az eredményeket JSON fájlba.
    """

    def __init__(self, directory_path=None):
        """Inicializálja a PngMetadataProcessor osztályt.

        Args:
            directory_path (str, optional): A feldolgozandó mappa elérési útja.
                                          Alapértelmezett érték: None.
        """
        self.file_handler = PngFileHandler(directory_path)

    def process_directory(self):
        """Feldolgozza a megadott mappában található összes PNG fájlt.

        A metódus végigmegy a megadott mappában található összes PNG fájlon,
        és összegyűjti azok metaadatait. A folyamat során:
        1. Azonosítja az érvényes PNG fájlokat
        2. Beolvassa minden fájl metaadatait
        3. Összesíti az eredményeket

        Returns:
            dict: Egy szótár, ahol a kulcsok a fájlnevek, az értékek pedig
                 a hozzájuk tartozó metaadatok szótár formátumban.
        """
        print(f"\nPNG fájlok keresése a következő mappában: {self.file_handler.directory_path}")
        print("=" * 70)

        png_files = self.file_handler.get_png_files()
        results = {}

        for file_path in png_files:
            metadata = MetadataReader.read_metadata(file_path)
            if metadata:
                results[os.path.basename(file_path)] = metadata

        if not results:
            print("\nNem találtunk érvényes PNG fájlokat a megadott mappában.")

        return results

    def save_results_to_json(self, results):
        """Elmenti az eredményeket JSON fájlba.

        Args:
            results (dict): A mentendő metaadatok szótár formátumban.

        Note:
            A fájl neve 'png_metadata_results.json' lesz, és az Image mappába kerül mentésre.
            Ha a fájl már létezik, felülírja azt.
        """
        if not results:
            print("\nNincs menthető adat.")
            return

        output_path = os.path.join(self.file_handler.directory_path, "png_metadata_results.json")
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            print(f"\nAz eredmények sikeresen mentve: {output_path}")
        except Exception as e:
            print(f"\nHiba történt a fájl mentése közben: {str(e)}")

# Példa használat és tesztelés
if __name__ == "__main__":
    # Példa a program használatára
    processor = PngMetadataProcessor("/content/drive/MyDrive/Image/")
    all_metadata = processor.process_directory()

    # Eredmények mentése JSON fájlba
    processor.save_results_to_json(all_metadata)

    # Összesítés
    if all_metadata:
        print("\nÖsszesítés:")
        print("=" * 70)
        print(f"Összesen {len(all_metadata)} érvényes PNG fájl metaadatait olvastuk be.")

## Pozitív prompt kiszedése

In [ ]:
import json
import os
import re

def find_longest_prompt_strings(json_file_path):
    """
    Finds the longest string value within the 'prompt' fields of a JSON file,
    specifically targeting the 'positive' field within the 'easy positive' node,
    FOR EACH IMAGE. Returns a dictionary mapping image filenames to longest strings.
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError, Exception) as e:
        print(f"Error: {type(e).__name__} - {e}")
        return {}  # Return an empty dictionary on error

    longest_strings = {}  # Dictionary to store results: {image_filename: longest_string}

    def find_in_prompt(node):
      # We use nonlocal within the inner function
      nonlocal longest_string
      if isinstance(node, dict):
        if node.get("class_type") == "easy positive" and "inputs" in node and "positive" in node["inputs"]:
          positive_prompt = node["inputs"]["positive"]
          if isinstance(positive_prompt, str) and len(positive_prompt) > len(longest_string):
            longest_string = positive_prompt
        else:
          for value in node.values():
            find_in_prompt(value)
      elif isinstance(node, list):
        for item in node:
          find_in_prompt(item)

    # Iterate through top-level keys (image filenames)
    for image_filename, image_data in data.items():
        longest_string = "" # Reset longest_string for EACH image
        if 'prompt' in image_data and isinstance(image_data['prompt'], str):
            try:
                prompt_data = json.loads(image_data['prompt'])
                find_in_prompt(prompt_data)  # Find longest in this image's prompt
                longest_strings[image_filename] = longest_string #Store result

            except json.JSONDecodeError:
                print(f"Invalid JSON in prompt for {image_filename}")
                #  Store empty string if prompt is invalid JSON
                longest_strings[image_filename] = ""

    return longest_strings


if __name__ == '__main__':
    file_path = "/content/drive/MyDrive/Image/png_metadata_results.json"
    if os.path.exists(file_path):
        result = find_longest_prompt_strings(file_path)
        if result:
            print("Longest strings in 'prompt' for each image:")
            for image_filename, longest_string in result.items():
                print(f"\n{image_filename}:\n{longest_string}") # kép: leghosszabb prompt
        else:
            print("No strings found in 'prompt' fields.")
    else:
        print(f"Error: File not found at {file_path}")
